<a href="https://colab.research.google.com/github/aditya-shriwastava/DeepRL_Experimentation/blob/master/PolicyGradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup the project in Google Colab

In [1]:
#@title Mount the project in Google Colab

GUEST = False
PROJECT = 'DeepRL_Experimentation'

import os
if GUEST:
  !git clone https://github.com/aditya-shriwastava/DeepRL_Experimentation.git
else:
  # Mount Google Drive
  from google.colab import drive
  drive.mount('/content/gdrive')

  # Setup Symlink to project folder
  DRIVE_PATH = '/content/gdrive/My\ Drive/' + PROJECT
  SYM_PATH = '/content/' + PROJECT
  if not os.path.exists(SYM_PATH):
    !ln -s $DRIVE_PATH $SYM_PATH

os.chdir( '/content/' + PROJECT)

Mounted at /content/gdrive


In [2]:
#@title apt install requirements

!apt update > /dev/null 2>&1
!apt install -y --no-install-recommends \
        xvfb \
        ffmpeg > /dev/null 2>&1

In [3]:
#@title pip install requirements

!pip install pyvirtualdisplay > /dev/null 2>&1
!pip3 install box2d-py > /dev/null 2>&1
!pip install Box2D > /dev/null 2>&1

In [4]:
#@title Setup Virtual Display

from pyvirtualdisplay import Display

display = Display(visible=0, size=(1400, 900))
display.start()

# Test if setup is completed and working properly

In [6]:
import gym
from colab_utils import (
    wrap_env,
    show_video
)

env = wrap_env(gym.make("CartPole-v1"))

observation = env.reset()
for i in range(100):
    env.render(mode='rgb_array')
    obs, rew, term, _ = env.step(env.action_space.sample() ) 
    if term:
      break;
            
env.close()
show_video()

# Policy Gradient Algorithm Experimentation

In [26]:
#@title Imports
from importlib import reload
import gym

import pg_agent
import MLP_policy
reload(MLP_policy)
reload(pg_agent)
from pg_agent import *

import utils
reload(utils)

<module 'utils' from '/content/gdrive/My Drive/DeepRL_Experimentation/utils.py'>

## REINFORCE Algorithm Experimentation

In [29]:
env = gym.make("CartPole-v1")

actor_params = {\
"ac_dim": env.action_space.n,\
"ob_dim": env.observation_space.shape[0],\
"n_layers": 5,\
"hidden_layer_size": 100,\
"learning_rate": 1e-4}

agent_params = {\
"actor_params": actor_params}

agent = REINFORCEAgent(agent_params)

print(f"E[rewards_sum] before training: {utils.expected_rewards_sum(env, agent.actor, env._max_episode_steps, 200)}")

# Training Loop
for _ in range(500):
  trajectories = utils.sample_trajectories(env, agent.actor, env._max_episode_steps, 5000)
  agent.train(trajectories)

print(f"E[rewards_sum] after training: {utils.expected_rewards_sum(env, agent.actor, env._max_episode_steps, 200)}")

E[rewards_sum] before training: 21.65
E[rewards_sum] after training: 437.65


In [31]:
from colab_utils import (
    wrap_env,
    show_video
)

env = wrap_env(env)

ob = env.reset()
for i in range(500):
    env.render(mode='rgb_array')
    ob, reward, done, _ = env.step(agent.actor.get_action(ob)) 
    if done:
      break;
            
env.close()
show_video()

## General Policy Gradient Algorithm Experimentation